In [1]:
import pandas as pd

In [2]:
dx=pd.read_csv('Thesis/DIME/dime_recipients_all_1979_2014.csv', \
low_memory=False, header=0, usecols=[3,24], names=['pacid', 'paccfscore'])

In [3]:
dx.columns.tolist()

['pacid', 'paccfscore']

In [4]:
dx = dx.drop_duplicates('pacid', keep='first')

dx = dx.dropna(subset = ['paccfscore'])

In [5]:
ds = pd.read_csv('pacidtocid.amount.sum')

In [6]:
ds.drop_duplicates(inplace=True, keep='first')

In [7]:
mrg = pd.merge(dx, ds, on = 'pacid', how='right')

In [8]:
mrg.drop_duplicates(inplace=True, keep='first')

In [9]:
dz = mrg[(mrg['amount'] > 0)]

In [10]:
len(mrg)

1200858

In [11]:
mrg.count()

pacid         1200858
paccfscore     986270
cycle         1200858
cid           1200858
amount        1200858
date          1200858
catcode       1200858
feccandid     1200858
loan          1200858
dtype: int64

In [12]:
dff=mrg[['feccandid', 'cid','date','pacid','amount', ]].copy()

dff['pacamtsum'] = dff.groupby(['date', 'pacid', 'cid']).\
amount.transform('sum')
dff.drop(['amount'], axis=1, inplace=True)
dff.drop_duplicates(inplace=True)

In [13]:
df=mrg[['cid','date','amount']].copy()

df['catamtsum']=df.groupby(['date', 'cid']).\
amount.transform('sum')
df.drop(['amount'], axis=1, inplace=True)
df.drop_duplicates(inplace=True)

In [14]:
dffmerge=pd.merge(dff, df, how='left', on=['cid', 'date'], sort=True)
dffmerge.drop_duplicates(inplace=True)

In [15]:
dq=mrg[['paccfscore','pacid']].copy()#feccandcfscore','pacfscore','catcode', 'feccandid', 'cycle', \
dq.drop_duplicates(inplace=True)#'loan'], axis=1, inplace=True)

In [16]:
dq.pacid.nunique()

8437

In [17]:
dq.isnull().sum()

paccfscore    3193
pacid            0
dtype: int64

In [18]:
dffmerge=pd.merge(dffmerge, dq, how='left', on='pacid', sort=True)
dffmerge.drop_duplicates(inplace=True)

In [19]:
dffmerge.dropna(subset=(['paccfscore']), inplace=True)

In [20]:
dffmerge['stdv_paccfscore']=dffmerge.groupby(['date', 'cid']).\
paccfscore.transform('std')


In [21]:
dffmerge.count()

feccandid          751112
cid                751112
date               751112
pacid              751112
pacamtsum          751112
catamtsum          751112
paccfscore         751112
stdv_paccfscore    748757
dtype: int64

In [22]:
dffmerge.head(1)

,feccandid,cid,date,pacid,pacamtsum,catamtsum,paccfscore,stdv_paccfscore
0,H2CT06014,N00000561,2005,C00000042,1000,605910,0.782,0.290863


In [23]:
dffmerge['catcode_cfscore'] = (dffmerge.pacamtsum / \
dffmerge.catamtsum) * dffmerge.paccfscore


In [24]:
dffmerge.drop_duplicates(['date', 'cid', 'pacamtsum'],keep='first',  inplace=True)

In [25]:
dffmerge['catcode_cfscorewtd'] = dffmerge.groupby(['date', 'cid']).\
catcode_cfscore.transform('sum')

In [26]:
di = pd.read_csv('Thesis/DIME/dime_recipients_all_1979_2014.csv',\
header=0, usecols=[1,3,23,25], names=['date', 'feccandid', 'feccandcfscore', 'feccandcfscore.dyn']) 
di = di.dropna(subset = ['feccandcfscore'])
di = di.dropna(subset = ['feccandcfscore.dyn'])
di = di.drop_duplicates('feccandid', keep='first')


In [27]:
di.feccandid.nunique()

88260

In [28]:
dffmerge= pd.merge(di, dffmerge, on=['feccandid', 'date'], how='right')
#dffmerge.drop_duplicates(inplace=True, keep='first')

In [29]:
dffmerge['feccandcfscore.dyn'].fillna(dffmerge.feccandcfscore, inplace=True)## by employing this line\
## line of code I am able to remove 1383 NaN values

dffmerge['catcode_cfscoremean']=dffmerge.groupby(['date', 'cid']).catcode_cfscorewtd.transform('mean')

dffmerge['cfcontrol']=(dffmerge['catcode_cfscoremean']-dffmerge['feccandcfscore.dyn'])

In [30]:
dffmerge.head()

,date,feccandid,feccandcfscore,feccandcfscore.dyn,cid,pacid,pacamtsum,catamtsum,paccfscore,stdv_paccfscore,catcode_cfscore,catcode_cfscorewtd,catcode_cfscoremean,cfcontrol
0,2006.0,S6WY00183,-0.549,-0.544,N00027685,C00003558,5000,7000,-0.677,0.199786,-0.483571,-0.606143,-0.606143,-0.062143
1,2006.0,S6WY00183,-0.549,-0.544,N00027685,C00027359,1000,7000,-0.858,0.199786,-0.122571,-0.606143,-0.606143,-0.062143
2,2006.0,S6WI00129,-1.818,-1.769,N00027684,C00428664,201,536,-1.683,NaN,-0.631125,-0.631125,-0.631125,1.137875
3,2006.0,S6WA00263,-1.527,-1.436,N00028623,C00114439,4610,4610,-1.070,NaN,-1.070000,-1.070000,-1.070000,0.366000
4,2006.0,S6WA00255,-1.730,-1.662,N00028059,C00361493,5120,5321,-1.675,0.005657,-1.611727,-1.675302,-1.675302,-0.013302


In [31]:
dffmerge.drop(['feccandid', 'feccandcfscore.dyn', 'feccandcfscore', \
'pacid', 'pacamtsum', 'catamtsum', 'paccfscore',  \
'catcode_cfscore', 'catcode_cfscorewtd', 'catcode_cfscoremean'], \
              axis=1, inplace=True)

In [32]:
dffmerge.head(1)

,date,cid,stdv_paccfscore,cfcontrol
0,2006.0,N00027685,0.199786,-0.062143


In [33]:
dffmerge.count()

date               117388
cid                117388
stdv_paccfscore    115033
cfcontrol           19969
dtype: int64

In [34]:
dffmerge.to_csv('crsp.dime.todf', index=False)

In [35]:
dffmerge.cid.nunique()

4040